In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3_gh11qs
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3_gh11qs
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10739 sha256=99634096e9ae99b658cd96f87c6e8ccf9cbc16bf9f7ab36b8a182687000c7e83
  Stored in directory: /tmp/pip-ephem-wheel-cache-gvqnh8e_/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
pip install nvcc4jupyter

In [4]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmps5ubwt1y".


In [5]:
%%cuda
#include <stdio.h>

#include <stdlib.h>

#include <time.h>

#include <cuda.h>

#define BLOCK_SIZE 16

__global__ void gpu_matrix_mult(int *a,int *b, int *c, int n)

{

int row = blockIdx.y * blockDim.y + threadIdx.y;

int col = blockIdx.x * blockDim.x + threadIdx.x;

int sum = 0;

if( col < n && row < n)

{

for(int i = 0; i < n; i++)

{

sum += a[row * n + i] * b[i * n + col];

}

c[row * n + col] = sum;

}

}

int main(int argc, char const *argv[])

{

int n;

for(n = 64; n <=8192 ; n *= 2) {

// allocate memory in host RAM, h_cc is used to store CPU result

int *h_a, *h_b, *h_c, *h_cc;

cudaMallocHost((void **) &h_a, sizeof(int)*n*n);

cudaMallocHost((void **) &h_b, sizeof(int)*n*n);

cudaMallocHost((void **) &h_c, sizeof(int)*n*n);

// initialize matrix A

for (int i = 0; i < n; ++i) {

for (int j = 0; j < n; ++j) {

h_a[i * n + j] = 2;

}

}

// initialize matrix B

for (int i = 0; i < n; ++i) {

for (int j = 0; j < n; ++j) {

h_b[i * n + j] = 3;

}

}

float naive_gpu_elapsed_time_ms;

// some events to count the execution time

//clock_t st, end;

cudaEvent_t start, stop;

cudaEventCreate(&start);

cudaEventCreate(&stop);

// Allocate memory space on the device

int *d_a, *d_b, *d_c;

cudaMalloc((void **) &d_a, sizeof(int)*n*n);

cudaMalloc((void **) &d_b, sizeof(int)*n*n);

cudaMalloc((void **) &d_c, sizeof(int)*n*n);

// copy matrix A and B from host to device memory

cudaMemcpy(d_a, h_a, sizeof(int)*n*n, cudaMemcpyHostToDevice);

cudaMemcpy(d_b, h_b, sizeof(int)*n*n, cudaMemcpyHostToDevice);

unsigned int grid_rows = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;

unsigned int grid_cols = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;

dim3 dimGrid(grid_cols, grid_rows);

dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

cudaEventRecord(start, 0);

gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, n);

cudaThreadSynchronize();

// time counting terminate

cudaEventRecord(stop, 0);

cudaEventSynchronize(stop);

// Transfer results from device to host

cudaMemcpy(h_cc, d_c, sizeof(int)*n*n, cudaMemcpyDeviceToHost);

// compute time elapsed on GPU computing

cudaEventElapsedTime(&naive_gpu_elapsed_time_ms, start, stop);

printf("Time elapsed on naive GPU matrix multiplication of %dx%d . %dx%d : %f ms.\n\n", n, n, n, n, naive_gpu_elapsed_time_ms);

// free memory

cudaFree(d_a);

cudaFree(d_b);

cudaFree(d_c);



}

return 0;

}

Time elapsed on naive GPU matrix multiplication of 64x64 . 64x64 : 121.443390 ms.

Time elapsed on naive GPU matrix multiplication of 128x128 . 128x128 : 0.045760 ms.

Time elapsed on naive GPU matrix multiplication of 256x256 . 256x256 : 0.170304 ms.

Time elapsed on naive GPU matrix multiplication of 512x512 . 512x512 : 1.152736 ms.

Time elapsed on naive GPU matrix multiplication of 1024x1024 . 1024x1024 : 9.113280 ms.

Time elapsed on naive GPU matrix multiplication of 2048x2048 . 2048x2048 : 74.949921 ms.

Time elapsed on naive GPU matrix multiplication of 4096x4096 . 4096x4096 : 384.109467 ms.

Time elapsed on naive GPU matrix multiplication of 8192x8192 . 8192x8192 : 2269.988770 ms.


